In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers


In [2]:
from google.colab import files
uploaded = files.upload()


In [3]:
titanic_file_path = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")


30874/30874 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
df = pd.read_csv(titanic_file_path)
print(df.head())


   survived     sex   age  n_siblings_spouses  parch     fare  class     deck  \
0         0    male  22.0                   1      0   7.2500  Third  unknown   
1         1  female  38.0                   1      0  71.2833  First        C   
2         1  female  26.0                   0      0   7.9250  Third  unknown   
3         1  female  35.0                   1      0  53.1000  First        C   
4         0    male  28.0                   0      0   8.4583  Third  unknown   

   embark_town alone  
0  Southampton     n  
1    Cherbourg     n  
2  Southampton     y  
3  Southampton     n  
4   Queenstown     y  


In [5]:
# Example features
titanic_features_dict = {
    'age': np.array([22, 38, 26, 35]),  # Sample feature values
    'fare': np.array([7.25, 71.2833, 7.925, 8.05])
}
titanic_labels = np.array([0, 1, 1, 0])  # Sample labels

# Create a dataset
titanic_ds = tf.data.Dataset.from_tensor_slices((titanic_features_dict, titanic_labels))


In [6]:
titanic_batches = titanic_ds.shuffle(len(titanic_labels)).batch(32)


In [7]:
def titanic_model(preprocessing_head, inputs):
    body = tf.keras.Sequential([
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    preprocessed_inputs = preprocessing_head(inputs)
    result = body(preprocessed_inputs)
    model = tf.keras.Model(inputs, result)

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam())
    return model


In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
import numpy as np

# Example preprocessing function
def titanic_preprocessing(inputs):
    # Normalize the inputs (assuming inputs are TensorFlow Tensors)
    return inputs / 255.0  # Adjust this to fit the actual dataset preprocessing

# Define the model-building function
def titanic_model(preprocessing_head, input_shape):
    inputs = layers.Input(shape=input_shape)  # Define the input layer
    preprocessed_inputs = preprocessing_head(inputs)
    body = tf.keras.Sequential([
        layers.Dense(64, activation='relu'),
        layers.Dense(1)  # Output layer for binary classification
    ])

    result = body(preprocessed_inputs)

    # Create and compile the model
    model = tf.keras.Model(inputs=inputs, outputs=result)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer=tf.keras.optimizers.Adam())
    return model

# Example input data (replace with Titanic dataset)
inputs = pd.DataFrame({
    'age': [22, 38, 26, 35],  # Example features
    'fare': [7.25, 71.2833, 7.925, 53.1],
})

# Convert Pandas DataFrame to TensorFlow Tensor
inputs_tensor = tf.convert_to_tensor(inputs, dtype=tf.float32)

# Create an instance of the model with the shape of the inputs
input_shape = inputs_tensor.shape[1:]  # Get the shape of the inputs (number of features)
titanic_model_instance = titanic_model(titanic_preprocessing, input_shape)

# Example labels (replace with actual Titanic labels)
titanic_labels = [0, 1, 1, 0]

# Convert labels to TensorFlow Tensor
titanic_labels_tensor = tf.convert_to_tensor(titanic_labels, dtype=tf.float32)

# Create a dataset with the features and labels
titanic_ds = tf.data.Dataset.from_tensor_slices((inputs_tensor, titanic_labels_tensor))

# Shuffle and batch the dataset
titanic_batches = titanic_ds.shuffle(len(titanic_labels)).batch(32)

# Train the model
titanic_model_instance.fit(titanic_batches, epochs=5)


Epoch 1/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 0.6902
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 0.6899
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6898
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 0.6896
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 0.6894


In [14]:
# Evaluate the model on the test data
test_loss = titanic_model_instance.evaluate(test_inputs_tensor, test_labels_tensor)

# Output the loss
print(f"Test Loss: {test_loss}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step - loss: 0.7064
Test Loss: 0.7064244151115417


In [15]:
titanic_model_instance.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy']  # This ensures accuracy is included as a metric
)

# Evaluate the model on the test data
test_loss, test_accuracy = titanic_model_instance.evaluate(test_inputs_tensor, test_labels_tensor)

# Output both loss and accuracy
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step - accuracy: 0.3333 - loss: 0.7064
Test Loss: 0.7064244151115417, Test Accuracy: 0.3333333432674408
